In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score, f1_score

In [ ]:
# Load the data
#train_df = pd.read_csv('/content/KCDH2024_Test_GroundTruth.csv')
#test_df = pd.read_csv('/content/KCDH2024_Test_GroundTruth.csv')


train_df = pd.read_excel('/content/KCDH2024_Test_GroundTruth.csv', sheet_name=None)
test_df = pd.read_excel('/content/KCDH2024_Test_GroundTruth.csv', sheet_name=None)

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
# Preprocessing function
def preprocess_image(img_path, target_size=(224, 224)):
    img = cv2.imread(img_path)
    img = cv2.resize(img, target_size)
    img = img / 255.0
    return img

In [ ]:
# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/content/KCDH2024_Train_Input',  # Path to the training images directory
    target_size=(224, 224),     # Resize images to 224x224 pixels
    batch_size=32,              # Number of images to return in each batch
    class_mode='categorical'    # For multi-class classification
)

In [ ]:
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=1000,
    class_mode='categorical'
)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    '/content/KCDH2024_Test_Input',  # Path to the training images directory
    target_size=(224, 224),     # Resize images to 224x224 pixels
    batch_size=1000,              # Number of images to return in each batch
    class_mode='categorical'    # For multi-class classification
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

# Create the model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=50,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:


# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
preds = model.predict(test_generator)

# Compute additional metrics
test_labels = test_df['label'].values
pred_labels = np.argmax(preds, axis=1)
auc = roc_auc_score(test_labels, preds, multi_class='ovo')
map = average_precision_score(test_labels, preds, average='macro')
f1 = f1_score(test_labels, pred_labels, average='macro')

print(f"Accuracy: {test_acc}")
print(f"AUC: {auc}")
print(f"Mean Average Precision: {map}")
print(f"F1 Score: {f1}")

# Save the model
model.save('dermoscopic_lesion_model.h5')
